In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd

#diretorio = r'Bases\Base_atualizada/'
diretorio = r'C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\Manipulacao_arquivos\Manipulacao_Arquivos_Python\Bases/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
ESTABELE = ['CNPJ_BASE', 'CNPJ_ORDEM', 'CNPJ_DV', 'MATRIZ_FILIAL', 'NOME_FANTASIA',
       'SITUACAO_CADASTRAL', 'DATA_SITUACAO_CADASTRAL',
       'MOTIVO_SITUACAO_CADASTRAL', 'DATA_INICIO_ATIVIDADE', 'CNAE_PRINCIPAL',
       'CNAE_SECUNDARIO', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO',
       'COMPLEMENTO', 'BAIRRO', 'CEP', 'UF', 'MUNICIPIO', 'TELEFONE1',
       'TELEFONE2', 'FAX', 'EMAIL']

dtypes = {'CNPJ_BASE': 'category',
 'CNPJ_ORDEM': 'category',
 'CNPJ_DV': 'category'}

CNAES = {5611201:'Restaurantes e similares',
        5611203:'Lanchonetes casas de chá de sucos e similares',
        5611204:'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
        5611205:'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
        5612100: 'Serviços ambulantes de alimentação'}

lista_cnae = []
lista_descricao = []
for cnae in CNAES.keys():
    lista_cnae.append(cnae)
for descricao in CNAES.values():
    lista_descricao.append(descricao)
lista_descricao

In [ ]:
c = 0
for file in all_files:
    dados =pd.read_csv(f'{diretorio}{file}', names=ESTABELE, dtype=dtypes, sep=';')
    dados['CNAE_DESCRICAO'] = lista_descricao[c]
    dados = dados[[
        'CNPJ_BASE', 'CNPJ_ORDEM', 'CNPJ_DV', 'NOME_FANTASIA', 'CNAE_PRINCIPAL','CNAE_DESCRICAO',
        'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO',
       'COMPLEMENTO', 'BAIRRO', 'CEP', 'UF', 'MUNICIPIO', 'TELEFONE1']]
    c +=1
    dados.to_csv(f"base_csv_estabelecimentos/{file}", mode='w',index=False, sep=';')
dados.head(2)

In [ ]:
dados.columns

## Com Pandas

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
import glob

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

path = f'{diretorio_estabelecimentos}/*.csv'
all_files_estabelecimentos = glob.glob(path)

lista_df_estabelecimentos = []
for file in all_files_estabelecimentos:
    d_estabelecimento = pd.read_csv(file, 
    sep=';', 
    dtype=dtypes_ESTABELE, 
    names=ESTABELE)
    lista_df_estabelecimentos.append(d_estabelecimento)

estabelecimentos = pd.concat(lista_df_estabelecimentos, axis=0, ignore_index=True)

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

path = f'{diretorio_empresa}/*.csv'
all_files_empresa = glob.glob(path)

lista_df_empresas = []
for file in all_files_empresa:
    d_empresa = pd.read_csv(file,
        sep=';',
        dtype=dtypes_EMPRESA, 
        names=EMPRESA)
    #lista_df_empresas.append(d_empresa)

    #empresas = pd.concat(lista_df_empresas, axis=0, ignore_index=True)

    merged_data = pd.merge(estabelecimentos, d_empresa, on='CNPJ_BASE')

    chunk_size = 1000000
    for i in range(0, len(merged_data), chunk_size):
        df_chunk = merged_data.iloc[i:i+chunk_size]
        df_chunk.to_csv(f'merge_base/merged_data_{i}.csv', mode='a', index=False)

## Com Dask

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import dask.dataframe as dd
import pandas as pd

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

# Lendo os arquivos
estabelecimentos = dd.read_csv(diretorio_estabelecimentos + '/*.csv', delimiter=';',  dtype=dtypes_ESTABELE, names=ESTABELE)
empresas = dd.read_csv(diretorio_empresa + '/*.csv', delimiter=';', dtype=dtypes_EMPRESA, names=EMPRESA)

# unindo as tabelas
merged = estabelecimentos.merge(empresas, on='CNPJ_BASE')

merged.to_csv('merge_base/merged_data.csv', single_file=True)

# mostrando as primeiras linhas
print(merged.head())

## Com Vaex

In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
import vaex

# diretórios
diretorio_estabelecimentos = r'base_csv_empresas'
diretorio_empresa = r'base_csv_estabelecimentos'

# Parâmetros Estabelecimentos
ESTABELE = ['CNPJ_BASE','CNPJ_ORDEM','CNPJ_DV','NOME_FANTASIA','TIPO_LOGRADOURO','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO','CEP','UF','MUNICIPIO']
dtypes_ESTABELE = {'CNPJ_BASE': 'category',
'CNPJ_ORDEM': 'category',
'CNPJ_DV': 'category'}

# Parâmetros Empresas
EMPRESA = ['CNPJ_BASE', 'RAZAO_SOCIAL']
dtypes_EMPRESA = {'CNPJ_BASE': 'category'}

# Lendo os arquivos
estabelecimentos = vaex.open(diretorio_estabelecimentos + '/*.csv', delimiter=';')
empresas = vaex.open(diretorio_empresa + '/*.csv', delimiter=';')

# unindo as tabelas
merged = estabelecimentos.join(empresas, on='CNPJ_BASE')

# mostrando as primeiras linhas
print(merged.head())

In [ ]:
import pandas as pd
import json
dados = pd.read_csv(r'merge_base\merged_data_0.csv')
municipios = pd.read_csv('Municipios\municipios.csv', sep=';', names=['MUNICIPIO','CIDADE'])
dados = dados[['CNPJ_BASE', 'CNPJ', 'NOME_FANTASIA', 'RAZAO_SOCIAL', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO', 'COMPLEMENTO', 'BAIRRO',
       'CEP', 'UF', 'MUNICIPIO']]
merged_data = pd.merge(dados, municipios, on='MUNICIPIO')
merged_data.drop(columns='MUNICIPIO', inplace=True)
merged_data.to_csv(r'merge_base\merged_data_0.csv', index=False)


In [ ]:
#diretorio = r'Bases\Base_atualizada/'
import time
import pandas as pd
import json
base = pd.DataFrame()
diretorio = r'merge_base/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))
for file in all_files:
    dados = pd.read_csv(f'{diretorio}{file}', sep=';' )
    dados = dados.sort_values(by=['CNPJ','TELEFONE'], ascending=[True,False])
    dados['TELEFONE'] = dados['TELEFONE'].astype(str)
    dados['TELEFONE'] = dados['TELEFONE'].str.replace('.0', '', regex=True)
    dados['TELEFONE'] = dados['TELEFONE'].str.replace('(', '', regex=True)
    dados['TELEFONE'] = dados['TELEFONE'].str.replace(')', '', regex=True)
    base = pd.concat([base, dados[0:2000]], ignore_index=True)
base.to_csv("DB_SHOW_APPLE.csv",mode='w', index=False)
dados.head(2)
    

In [2]:
from tradutor import traduzir
import pandas as pd
dados =  pd.read_csv('apple-categories.csv')
print(dados.shape)
dados = dados.query('id.str.contains("food") or id.str.contains("restaurant")', engine='python')
#dados = dados.dropna(how='any')
print(dados.shape)


Using state Minas Gerais server backend.


(2135, 3)
(478, 3)


In [ ]:
dict_dados = {'novo_id':[],'novo_nome':[],'nova_descricao':[]}
for id in dados['id']:
    novo_id = traduzir(texto=id, idioma_origem='en', idioma_destino='pt')
    dict_dados['novo_id'].append(novo_id)

In [4]:
lista_nomes = []
for name in dados['name']:
    novo_nome = traduzir(name,'en', 'pt')
    lista_nomes.append(novo_nome)



Erro ao traduzir: Treinamento de segurança alimentar
Erro ao traduzir: Comida e supermercado
Erro ao traduzir: Tigelas de açaí
Erro ao traduzir: Padaria
Erro ao traduzir: Loja de bagels
Erro ao traduzir: Padaria
Erro ao traduzir: Patisserie
Erro ao traduzir: Loja de bebidas
Erro ao traduzir: Loja de bebidas
Erro ao traduzir: Fornecedor de refeições em caixa
Erro ao traduzir: Cervejaria
Erro ao traduzir: Brew Pub
Erro ao traduzir: Loja de chá de bolhas
Erro ao traduzir: Açougue
Erro ao traduzir: Loja de bolos
Erro ao traduzir: Chimney Cake Shop
Erro ao traduzir: Churriria
Erro ao traduzir: Barra de cidra
Erro ao traduzir: Café e café
Erro ao traduzir: Roastery de café
Erro ao traduzir: Loja de suprimentos de café e chá
Erro ao traduzir: Loja de conveniência
Erro ao traduzir: Spätkauf
Erro ao traduzir: CSA
Erro ao traduzir: Bolos de copo
Erro ao traduzir: Bolos personalizados
Erro ao traduzir: Delicatessen
Erro ao traduzir: Loja de sobremesas
Erro ao traduzir: Destilaria
Erro ao traduzir

In [3]:
lista_descricao = []
for descricao in dados['description']:
    nova_descricao = traduzir(descricao,'en', 'pt')
    lista_descricao.append(nova_descricao)

Erro ao traduzir: Empresas especializadas no fornecimento de cursos em segurança alimentar.
Erro ao traduzir: Empresas que vendem itens alimentares. Não inclui restaurantes.
Erro ao traduzir: Sementes de cânhamo. Embora as tigelas de açaí sejam geralmente doces, também existem versões salgadas contendo camarão ou peixe seco e tapioca (normalmente encontrado no norte do Brasil).
Erro ao traduzir: [Não use, use alimentos.Bakeries] Uma empresa vendendo assados.
Erro ao traduzir: Lojas que vendem bagels, principalmente.
Erro ao traduzir: Lojas que vendem assados, por exemplo pão, bolo e doces.
Erro ao traduzir: Uma padaria especializada em doces.
Erro ao traduzir: Lojas de varejo que vendem cerveja, vinho e bebidas espirituosas.
Erro ao traduzir: Um negócio que vende bebidas alcoólicas e não alcoólicas.
Erro ao traduzir: Negócios que fornecem refeições em caixa, podem ser para coleta ou entrega. Na Índia, isso é chamado de Tiffin e normalmente associado ao almoço.
Erro ao traduzir: Empresa

In [8]:
listadados = {'nomes': lista_nomes, 'descricao': lista_descricao}
novodados = pd.DataFrame(listadados)

novodados.to_excel('categorias_apple.xlsx')

In [3]:
nivel4 = pd.read_json('en_amostra_json\EN_db_SIGA_04_APPLE.json',orient='records')
nivel4.to_excel("nivel4.xlsx")

nivel5 = pd.read_json('en_amostra_json\EN_db_SIGA_05_APPLE.json',orient='records')
nivel5.to_excel("nivel5.xlsx")

In [4]:
novonivel4 = pd.read_excel('nivel4.xlsx', index_col=0)

novonivel5 = pd.read_excel('nivel5.xlsx', index_col=0)

In [5]:
novonivel4 = novonivel4.fillna('')
novonivel4.to_json('EN_db_SIGA_04_APPLE.json', orient='records')
novonivel4.head()

,EIN (CNPJ),CORPORATE_NAME,TRADING NAME,STREET,ADDRESS_NUMBER,ADDRESS_COMPLEMENT,DISTRICT,CITY,STATE,ZIP_CODE,LATITUDE,LONGITUDE,PHONE_NUMBER,SITE,CATEGORY_(CNAE),OPENING_HOURS,INSTAGRAM,FACEBOOK,SERVICE_OPTIONS,"('OPENING_HOURS', 'SERVICE_OPTIONS', 'INSTAGRAM', 'FACEBOOK')"
0,507391000135,P.A. VAROTO-LANCHES,QUIOSQUE ALTO ASTRAL,AVENIDA CASTELO BRANCO,33047,LJ 27 QUIOSQUE,JD GUILHERMINA,PRAIA GRANDE,SP,11724210,,,(13)349131,wwwQUIOSQUEALTOASTRAL.COM,Quiosque de comida,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,@quiosquealtoastral,https://pt-br.facebook.com/quiosquealtoastral,\nEntrega sem contato\nEntrega\nPara viagem\nR...,
1,14573000174,DEUSDETE EVANGELISTA DA SILVA,Indisponível,AVENIDA BEIRA MAR,12696,,BALNEARIO SAO MARTINHO,ILHA COMPRIDA,SP,11925000,,,(13)384238,wwwIndisponível.COM,Restaurante de comida caseira,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...,@indisponível,https://pt-br.facebook.com/indisponível,\nRetirada na porta\nEntrega sem contato\nEntr...,
2,265320000173,JOSE MARIA FELIX MIRACATU,BAR E MERCEARIA FELIX,RUA JACOB DO BANDOLIN,159,,JARDIM YOLANDA,MIRACATU,SP,11850000,,,(13)38473211,wwwBAREMERCEARIAFELIX.COM,Loja de conveniência,domingo \nFechado\n\nsegunda-feira\n11:00–15:0...,@baremerceariafelix,https://pt-br.facebook.com/baremerceariafelix,\nMesas externas\nEntrega sem contato\nEntrega...,
3,4758000106,JOSE GAMALIEL DE MORAES ME,"BAR, MERCEARIA E POUSADA MORAES",RUA EDWARD MANCIO,149,,ITAPEUNA,ELDORADO,SP,11960000,-24.478183,-48.309742,(13)38711312,"wwwBAR,MERCEARIAEPOUSADAMORAES.COM",Loja de conveniência,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...,"@bar,merceariaepousadamoraes","https://pt-br.facebook.com/bar,merceariaepousa...",\nPara viagem\nRefeição no local\n,
4,6217000290,ANITA JUANA DOMINGUES RODRIGUEZ,LA MACARENA II,AVENIDA CENTRAL,261,,GUARAU,PERUIBE,SP,11750000,-24.369146,-47.013984,(13)4554346,wwwLAMACARENAII.COM,Loja de massas,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...,@lamacarenaii,https://pt-br.facebook.com/lamacarenaii,\nEntrega sem contato\nEntrega\nPara viagem\nR...,


In [6]:
novonivel5 = novonivel5.fillna('')
novonivel5.to_json('EN_db_SIGA_05_APPLE.json', orient='records')
novonivel5.head()

,EIN (CNPJ),CORPORATE_NAME,TRADING NAME,STREET,ADDRESS_NUMBER,ADDRESS_COMPLEMENT,DISTRICT,CITY,STATE,ZIP_CODE,LATITUDE,LONGITUDE,PHONE_NUMBER,SITE,CATEGORY_(CNAE),OPENING_HOURS,INSTAGRAM,FACEBOOK,SERVICE_OPTIONS
0,4941979000153,L' ABBATE COMERCIAL LTDA,Indisponível,RUA WISARD,397,LOJA 02,VILA MADALENA,SAO PAULO,SP,5434080,-23.546496,-46.691124,(11),wwwIndisponível.COM,Cervejaria,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,@indisponível,https://pt-br.facebook.com/indisponível,\nMesas externas\nEntrega sem contato\nEntrega...
1,4260571000116,AKSA GOURMET LOUNGE E RESTAURANTE EIRELI,AKSA GOURMET LOUNGE,AVENIDA GUILHERME GIORGI,1050,,VILA CARRAO,SAO PAULO,SP,3422001,-23.549867,-46.53858,(11)045654,wwwAKSAGOURMETLOUNGE.COM,Cervejaria,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,@aksagourmetlounge,https://pt-br.facebook.com/aksagourmetlounge,\nMesas externas\nEntrega sem contato\nEntrega...
2,503749000151,EDILSON MARCELINO RIBEIRO ITU,Indisponível,AVENIDA PRUDENTE DE MORAES,468-A,,VILA NOVA,ITU,SP,13309050,-23.272899,-47.292044,(11)133821,wwwIndisponível.COM,Cervejaria,\ndomingo\nFechado\n\nsegunda-feira\n10:00–14:...,@indisponível,https://pt-br.facebook.com/indisponível,\nEntrega sem contato\nEntrega\nPara viagem\nR...
3,23030000113,GILMAR A FRARE,PADARIA MORUNGABA,PRAÇA JOAO PESSOA,142,,CENTRO,MORUNGABA,SP,13260000,-22.832452,-46.800462,(11)1477,wwwPADARIAMORUNGABA.COM,Cervejaria,domingo\nFechado\n\nsegunda-feira\n11:00–15:30...,@padariamorungaba,https://pt-br.facebook.com/padariamorungaba,\nRetirada na porta\nEntrega sem contato\nEntr...
4,15700000150,F. C. F. PESQUEIRO E RESTAURANTE LTDA,PESQUEIRO MURAYAMA,AVENIDA P.L. DO BRASIL,SM,KM 05,CENTRO,ARUJA,SP,7400630,-23.394437,-46.320446,(11)221497,wwwPESQUEIROMURAYAMA.COM,Cervejaria,domingo \nFechado\n\nsegunda-feira\n11:00–15:0...,@pesqueiromurayama,https://pt-br.facebook.com/pesqueiromurayama,\nMesas externas\nEntrega sem contato\nEntrega...
